# HW10
## Problem 1: Computational cost 

Input layer:  $O(d\cdot k)$  
Hidden layer: $ O(k\cdot k \cdot (q-1))$  
Output layer: $ O(k)$  

**In total: $O(d\cdot k + (q-1)\cdot k^2) + k) = O(d\cdot k + (q-1)\cdot k^2)) $**


## Problem 2: Neural networks with PyTorch

In [27]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F

Load CIFAR10 train and test set

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

100%|██████████| 170498071/170498071 [00:15<00:00, 10834371.34it/s]


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


1. Reducing the size of the training and test accordingly

In [26]:
# car: 1
# dog: 5
# ship: 8
classes = ('plane', 'car', 'bird', 'cat','deer', 'dog', 'frog', 'horse', 'ship', 'truck')

trainset = [ex for ex in trainset if ex[1]==1 or ex[1]==5 or ex[1]==8]
testset = [ex for ex in testset if ex[1]==1 or ex[1]==5 or ex[1]==8]

trainloader = DataLoader(trainset, batch_size=batch_size,shuffle=True)
testloader = DataLoader(testset, batch_size=batch_size,shuffle=False)

In [41]:
next(iter(trainloader))[0].shape

torch.Size([4, 3, 32, 32])

2. creat model

In [44]:
class Net(nn.Module):
    def __init__(self,inputdim):
        super(Net,self).__init__()
        self.fc1 = nn.Linear(inputdim,512)
        self.fc2 = nn.Linear(512,1)

    def forward(self,x):
        x = torch.flatten(x,1)
        x=F.relu(self.fc1(x))
        x=self.fc2(2)
        return x
    
net = Net(3*32*32)
net

Net(
  (fc1): Linear(in_features=3072, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=1, bias=True)
)

3. Optimizer and loss

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.paramters(),lr = 1e-3, momentum=0.9)
max_epochs = 10

for i in range(max_epochs):
    for i,data in next(iter(trainloader)):
        X,y = data
        
        # zero the gradient buffer
        optimizer.zero_grad()

In [36]:
x_test.flatten(1)

tensor([[-1.6063,  0.7648, -1.6090, -0.9929, -0.2790, -0.0833,  1.4728, -0.8165,
         -0.0702,  0.7762, -0.7074,  0.0476],
        [ 2.3348, -0.0199, -2.6846, -1.6174,  0.9966, -1.1365,  1.2314, -1.7660,
          0.9695, -1.6554, -0.1164, -0.1930]])